# Heirarchical Agglomerative Clustering HAC

Load iris dataset from sklearn.
```{python}
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
```
1. Implement HAC algorithm. Use the abstract class definition provided below. **(15 Points)**

2. Test your code first with uni-variate data as following; **(10 Points)**
```{python}
x = {'JAN':31.9, 'FEB':32.3, 'MAR':35, 'APR':52, 'MAY':60.8, 'JUN':68.7, 'JUL':73.3, 'AUG':72.1, 'SEP':65.2, 'OCT':54.8, 'NOV':40, 'DEC':38}
hac = HAC(param={'dist': 'eucl'})
hac.fit(x)
for c in hac.dendrogram:
    print(c)
```
Expected output:
```
(0, ['JAN', 'FEB'], 0.4)
(1, ['JUL', 'AUG'], 1.2)
(2, ['NOV', 'DEC'], 2.0)
(3, ['APR', 'OCT'], 2.8)
(4, ['JAN', 'FEB', 'MAR'], 2.9)
(5, ['JUN', 'SEP'], 3.5)
(6, ['APR', 'OCT', 'MAY'], 7.4)
```

2. Fit the HAC model to iris dataset. Print the heirarchy of clusters creatively. It need not to be a dendrogram but you can use [sklearn implementation](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py) for comparison. **(15 Points)**


In [403]:
from decimal import Decimal
import functools
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import minkowski


# Represents a cluster of points
class Cluster:
    def __init__(self, index, label, X, id):
        self.X = X.to_numpy()
        self.indexes = np.array([index]) 
        self.labels = np.array([label])
        self.id = id
        

    # returns the mean X vector for the cluster
    def getMeanX(self):
        return self.X.mean(0)
    
    # Add a member to the cluster
    def mergeCluster(self, cluster):
        self.X = np.concatenate((self.X, cluster.X), axis=0)
        self.indexes = np.concatenate((self.indexes, cluster.indexes))
        self.labels = np.concatenate((self.labels, cluster.labels))
        
    # # update the id for this cluster 
    # def updateId(self, id):
    #     self.id = id
        
class HAC:
    def __init__(self, param, X):
        self.param = param['dist']
        self.nodeList = NodeList()
        self.clusters = []
        if type(X) == dict:
            self.X = pd.DataFrame(data=list(X.values()))
            self.y = pd.Series(data=list(X.keys())).astype(str)
        elif isinstance(X, pd.DataFrame): # X is a dataframe
            self.X = X.iloc[:,:-1]
            self.y = X.iloc[:,-1].astype(str)
        else:
            raise Exception("Not a valid data type")
        
        self.makeClusters()
        self.__distances__()
        
    # Make initial clusters = one for every instance in self.X
    def makeClusters(self):
       for i, x in self.X.iterrows():
           self.clusters.append(Cluster(i, self.y.iloc[i], x, i))
           
    # get the distance bewteen two clusters    
    def getDistance(self, c1, c2):
        if self.param == 'eucl':
            return np.linalg.norm(np.subtract(c1.getMeanX(), c2.getMeanX())) # Euclidian distance between two points
        elif self.param == 'manh':
            return np.sum(np.abs(c1.getMeanX()-c2.getMeanX()))
        elif self.param == 'misk': 
            return minkowski(c1.getMeanX(), c2.getMeanX())
        else:
            raise Exception('Not a valid dist measure. Choose among eucl, manh, misk')

    # Iterate through all clusters and updates self.C with the pair-wise distances
    def __distances__(self):
        # initialize self.C as npmy array of size clusters x clusters
        self.C = np.zeros([len(self.clusters), len(self.clusters)])
        # Iterate through each cluster, and update self.C matrix with pairwise distances    
        for i1 in range(len(self.clusters)):
            for i2 in range(len(self.clusters)):
                self.C[i1, i2] = self.getDistance(self.clusters[i1], self.clusters[i2])
                

    # returns the indexes of the smallest distance from self.C
    def getClosestClusters(self):
        minIndexes = [0, 1]
        for i1 in range(len(self.C)):
            for i2 in range(len(self.C)):
                if self.C[i1,i2] > 0 and self.C[i1,i2] < self.C[minIndexes[0], minIndexes[1]] :
                    minIndexes = [i1, i2]
        return minIndexes
    
        
    # Merge the nearest clusters
    def __merge__(self):
        count = 0
        while len(self.clusters) > 1:
            # print(count)
            if count > 150:
                raise Exception("Preventing too many iterations")
            # 1. Find the cloest two clusters - cluster1 and cluster2
            [i1, i2] = self.getClosestClusters()
            c1, c2 = self.clusters[i1], self.clusters[i2]
            # 2. Merge the cluster1 into cluster2
            c2.mergeCluster(c1)
            # 3. Add item to self.dendrogram
            self.dendrogram.append((count, c2.indexes, self.C[i1,i2]))
            # 4. Remove cluster 1 from self.clusters, destroy it
            self.clusters.pop(i1)
            # 5. Call __distances__() again
            self.__distances__()
            count += 1
        
    # # Drawing Helper: returns a list of the indexes in order of when they were clustered  
    # def getIndexClusterSequence(self):
    #     seq = []
    #     for d in self.dendrogram:
    #         for x in d[1]:
    #             if x not in seq:
    #                 seq.append(x)
    #     return seq
    
    
    # Splits the indices in the cluster into either seenIndices (already in self.nodeList) or newIndices
    def getIndexSplit(self, cluster):
        newIndices = []
        seenIndices = []
        for i in cluster[1]:
            if i not in self.nodeList.indexes:
                newIndices.append(i)
            else:
                seenIndices.append(i)
        return newIndices, seenIndices

    # Returns a list of 1-2 nodes from self.NodeList with the largest distance that contains at least one element from the indexList parameter
    def getChildNodes(self, indexList):
        maxNodes = [None, None]
        maxDist = [0, 0]
        nodes = self.nodeList.nodes.copy()
        # Get the largest node
        for k in range(2): # repeat twice
            if len(nodes) > 0:
                for node in nodes: # Look at each node
                    for i in indexList: # Look at each index
                        if i in node.indices and node.dist > maxDist[k]:
                            maxNodes[k] = node
                            maxDist[k] = node.dist
                if maxNodes[k] != None:
                    nodes.remove(maxNodes[k])
            else:
                maxNodes[k] = None            
        return maxNodes[0], maxNodes[1]

    # Build the nodelist
    def buildNodeList(self):
        for cluster in self.dendrogram:
            # For the new cluster, find the indices that are new vs. seen
            newIndices, seenIndices = self.getIndexSplit(cluster)
            newNode = Node(cluster[1], newIndices, cluster[2])
            
            # Case 1 - two new indexes -> new node will have no children
            if len(newIndices) == 2:
                # Set no children
                pass
            
            # Case 2 - one new index -> new node will have one chlild
            elif len(newIndices) == 1:
                child1, child2 = self.getChildNodes(cluster[1])
                if child1 != None:
                    child1.parent = newNode
                    newNode.children.append(child1)
                
            # Case 3 - no new index -> new node will have two chlildren
            elif len(newIndices) == 0 and len(seenIndices) > 0:
                child1, child2 = self.getChildNodes(cluster[1])
                if child1 != None:
                    child1.parent = newNode
                    newNode.children.append(child1)
                if child2 != None:
                    child2.parent = newNode
                    newNode.children.append(child2)

            self.nodeList.addNode(newNode)

    
    
    '''Creatively draw the dendrogram'''
    # Draw the nodelist
    def drawDendrogram(self):
        self.buildNodeList()
        self.nodeList.getRoot().printNode("  ")
        

    def fit(self):
        self.__distances__()
        self.dendrogram = list()
        self.__merge__()


# Helper class to visualize the heirarchy as a hierarchy of nodes
class Node:
    def __init__(self, allIndices, newIndices, dist):
        self.parent = None
        self.children = []
        self.indices = allIndices
        self.newIndices = newIndices
        self.dist = dist
        
    # Recursive function to print the node information
    def printNode(self, space):
        if self.children == []:
            print(space, "Dist:", round(self.dist,2), ", Indices:", self.indices)    
        else:
            print(space, "Dist:", round(self.dist,2), ", Indices:", self.indices, ", sub-clusters: {")    
            for child in self.children:
                child.printNode(space + space)
            print(space, "}")
    
    
# Helper class to visualize the heirarchy as a hierarchy of nodes
class NodeList:
    def __init__(self):
        self.nodes = []
        self.indexes = []
        
    def addNode(self, node):
        self.nodes.append(node)
        for i in range(len(node.newIndices)):
            self.indexes.append(node.newIndices[i])
        
    def getRoot(self):
        maxNode = None
        maxDist = 0
        for node in self.nodes:
            if node.dist > maxDist:
                maxDist = node.dist
                maxNode = node
        return maxNode


In [387]:
from pprint import pprint
x = {'JAN':31.9, 'FEB':32.3, 'MAR':35, 'APR':52, 'MAY':60.8, 'JUN':68.7, 'JUL':73.3, 'AUG':72.1, 'SEP':65.2, 'OCT':54.8, 'NOV':40, 'DEC':38}
y = HAC({'dist': 'eucl'}, x)

y.fit()
for d in y.dendrogram:
    print(d)

# NOTE: Indexes are shown below ... {0=JAN, 1=FEB, ... 11=DEC}

(0, array([1, 0]), 0.3999999999999986)
(1, array([7, 6]), 1.2000000000000028)
(2, array([11, 10]), 2.0)
(3, array([9, 3]), 2.799999999999997)
(4, array([2, 1, 0]), 2.9000000000000057)
(5, array([8, 5]), 3.5)
(6, array([8, 5, 7, 6]), 5.749999999999986)
(7, array([11, 10,  2,  1,  0]), 5.933333333333337)
(8, array([9, 3, 4]), 7.399999999999999)
(9, array([9, 3, 4, 8, 5, 7, 6]), 13.958333333333336)
(10, array([11, 10,  2,  1,  0,  9,  3,  4,  8,  5,  7,  6]), 28.402857142857144)


In [369]:
''' Test printing the heirarchy in creative way on the monthly temps dataset'''
y.drawDendrogram()


   Dist: 28.4 , Indices: [11 10  2  1  0  9  3  4  8  5  7  6] , sub-clusters: {
     Dist: 13.96 , Indices: [9 3 4 8 5 7 6] , sub-clusters: {
         Dist: 7.4 , Indices: [9 3 4] , sub-clusters: {
                 Dist: 2.8 , Indices: [9 3]
         }
         Dist: 5.75 , Indices: [8 5 7 6] , sub-clusters: {
                 Dist: 3.5 , Indices: [8 5]
                 Dist: 1.2 , Indices: [7 6]
         }
     }
     Dist: 7.4 , Indices: [9 3 4] , sub-clusters: {
         Dist: 2.8 , Indices: [9 3]
     }
   }


In [370]:
'''Fit to the Iris Dataset'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

iris = datasets.load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])


z = HAC({'dist': 'eucl'}, df)
z.fit()

In [371]:
# Implement __display__ method to cretively show the contents of dendrogram. 

z.drawDendrogram()

   Dist: 1.41 , Indices: [149 127  76  56 146 123  70 133 138 134  51  65 126 142 121  83  54  58
  75  85 101 113  72  63  91 147 103 137 116 110 108 128 114  86 111  50
  52  77 119  97  66  61  78  74  73  96  95  71  87  84  68  55 106  92
  90  67  82  99  88  94  89  64  69  53  62  59  81  80  79 148 141 112
 104 139 145 115 132 140 136 129 124 144 143 130 125 100 102 120 107 135
 122 118 109 105 131 117  98  93  57  60  18   5  44  33  14  15  49  40
   7  37  11  39  28  24  17   0   4  48  46  31  20  21  19  43  32  10
  16  36  23  27  26  47  22  12   3   2  45  35   9   1  34  30  29   6
  25  42   8  38  41  13] , sub-clusters: {
     Dist: 0.81 , Indices: [149 127  76  56 146 123  70 133 138 134  51  65 126 142 121  83  54  58
  75  85 101 113  72  63  91 147 103 137 116 110 108 128 114  86 111  50
  52  77 119  97  66  61  78  74  73  96  95  71  87  84  68  55 106  92
  90  67  82  99  88  94  89  64  69  53  62  59  81  80  79 148 141 112
 104 139 145 115 132 140 136